# 당뇨병 발병 예측


최근 수년 동안 헬스케어 머신러닝이 발전하면서 분석에 쓸 수 있는 큰 데이터셋들이 수집, 관리되고 있다. 신경망 기술을 사용하면 질병이 발생하기 전에 다양한 질병의 발병을 예측할 수 있게 되었다. 이 장에서는 딥러닝과 그리드 탐색(grid search) 방법을 사용하여 당뇨병 발병을 예측하는 사례를 소개한다. 딥러닝이 무엇인지 소개하고, 최적화를 위한 파라미터와 적절한 파라미터를 선택하는 방법도 다룬다.

다음과 같은 순서로 설명한다.

- 딥러닝, 그리드 탐색을 사용한 당뇨병 진단 
- 데이터셋 소개
- 케라스 모델 설계 
- 사이킷-런으로 그리드 탐색 실행하기 
- 드롭아웃 규제로 과대적합 줄이기 
- 최적의 초매개변수 찾기 
- 최적 초매개변수를 사용하여 예측하기 

## 딥러닝과 그리드 탐색을 사용한 당뇨병 진단 

이 장에서는 딥러닝 알고리즘을 사용하여 당뇨병 발생을 예측하는 방법과 그리드 탐색법을 사용하여 알고리즘을 최적화하는 방법에 대해서 다룬다.

In [1]:
import sys
import pandas
import numpy
import sklearn
import keras

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Python: 3.8.6 | packaged by conda-forge | (default, Jan 25 2021, 23:22:12) 
[Clang 11.0.1 ]
Pandas: 1.2.4
Numpy: 1.20.2
Sklearn: 0.24.2
Keras: 2.4.3


넘파이와 판다스 패키지를 임포트한다. 

In [2]:
import numpy as np
import pandas as pd

## 데이터셋 소개

여기서 사용할 피마 인디언 당뇨병 데이터셋을 임포트한다. 이 데이터셋에는 750명 환자에 대한 상세 정보가 담겨있다.

데이터는 <https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv>에 있다. 이 URL로 찾아가 보면 여러 원 정보들을 확인할 수 있다. 데이터셋을 임포트하고서 열의 이름을 지정한다. 다음 `pd.read_csv()` 함수를 사용하여 데이터셋을 판다스 데이터프레임으로 읽어온다. 

In [3]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI', 'pedigree_function', 'age', 'class']

df = pd.read_csv(url, names=names)

In [4]:
# 기술 통계 확인
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## 데이터 전처리

데이터를 보고 결측값들이 어떻게 들어가 있는지 확인하자. 

다음 코드를 사용하여 혈당값이 0인 행들을 필터링해 보자.

In [5]:
df[df['glucose_concentration'] == 0]

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


먼저 결측값을 `NaN`(`np.nan`) 값으로 코딩한 후에, 결측값이 있는 행들을 제거하려고 한다. 이렇게 하기 위해서 검색 대상 열을 정의한다. `n_prgenancy`, `age`, `class`를 제외한 열들을 선택한다. 

In [6]:
columns = ['glucose_concentration', 'blood_pressure (mm Hg)', 
           'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

In [7]:
for col in columns:
    df[col].replace(0, np.nan, inplace=True)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000



이제 결측값을 모두 `NaN`으로 처리했다. 이제 결측값을 가진 사례들을 다음과 같은 코드로 제거한다.


In [8]:
df.dropna(inplace=True)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
# 데이터프레임을 numpy 배열로 변환 
dataset = df.values
print(dataset.shape)

(392, 9)


In [10]:
X = dataset[:, 0:8]
Y = dataset[:, 8].astype(int)

In [11]:
print(X.shape)
print(Y.shape)
print(X[:5])
print(Y[:5])

(392, 8)
(392,)
[[1.000e+00 8.900e+01 6.600e+01 2.300e+01 9.400e+01 2.810e+01 1.670e-01
  2.100e+01]
 [0.000e+00 1.370e+02 4.000e+01 3.500e+01 1.680e+02 4.310e+01 2.288e+00
  3.300e+01]
 [3.000e+00 7.800e+01 5.000e+01 3.200e+01 8.800e+01 3.100e+01 2.480e-01
  2.600e+01]
 [2.000e+00 1.970e+02 7.000e+01 4.500e+01 5.430e+02 3.050e+01 1.580e-01
  5.300e+01]
 [1.000e+00 1.890e+02 6.000e+01 2.300e+01 8.460e+02 3.010e+01 3.980e-01
  5.900e+01]]
[0 1 1 1 1]


## 데이터 정규화

여기서는 데이터 정규화를 시킨다.

데이터 정규화는 `sklearn` 라이브러리가 제공하는 `StandardScaler`를 사용한다.

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)

In [13]:
# 훈련 데이터의 변환과 출력
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-9.063045e-18,1.132881e-17,-4.531523e-16,1.087565e-16,1.064908e-16,1.631348e-16,1.812609e-17,1.110223e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


## 케라스 모델 정의

필요한 패키지와 레이어를 임포트한다.

In [14]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [15]:
def create_model():
     #  케라스 모델 정의 
     model = Sequential()
     model.add(Dense(8, input_dim = 8, 
                    kernel_initializer='normal', activation='relu'))
     model.add(Dense(4, input_dim = 8, 
                    kernel_initializer='normal', activation='relu'))
     model.add(Dense(1, activation='sigmoid'))
 
     # 모델 컴파일 
     adam = Adam(lr = 0.01)
     model.compile(loss = 'binary_crossentropy', 
                  optimizer = adam, metrics = ['accuracy'])
     return model

In [16]:
model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


## 사이킷런을 사용하여 그리드 탐색법 수행 

In [17]:
# 최적 배치 크기와 에포크를 정하기 위한 그리드 탐색
# 랜덤 시드 정의
seed = 6
np.random.seed(seed)

# 모델 정의
def create_model():
    # 케라스 모델 생성
    model = Sequential()
    model.add(Dense(8, input_dim = 8, 
                    kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, 
                    kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일 
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = adam, 
                  metrics = ['accuracy'])
    return model

# 모델 생성
model = KerasClassifier(build_fn = create_model, verbose = 1)

# 그리드 탐색 매개변수 정의 
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# 그리드 탐색 매개변수를 딕셔너리로 만들기 
param_grid = dict(batch_size=batch_size, epochs=epochs)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model, param_grid = param_grid, 
                    cv = KFold(random_state=seed, shuffle=True), 
                    verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=10....................................
Epoch 1/10
32/32 [==============================] - 0s 750us/step - loss: 0.6828 - accuracy: 0.6875
Epoch 2/10
32/32 [==============================] - 0s 735us/step - loss: 0.6583 - accuracy: 0.6716
Epoch 3/10
32/32 [==============================] - 0s 709us/step - loss: 0.6459 - accuracy: 0.6669
Epoch 4/10
32/32 [==============================] - 0s 733us/step - loss: 0.6408 - accuracy: 0.6638
Epoch 5/10
32/32 [==============================] - 0s 734us/step - loss: 0.6289 - accuracy: 0.6804
Epoch 6/10
32/32 [==============================] - 0s 738us/step - loss: 0.6401 - accuracy: 0.6617
Epoch 7/10
32/32 [==============================] - 0s 688us/step - loss: 0.6208 - accuracy: 0.6891
Epoch 8/10
32/32 [==============================] - 0s 708us/step - loss: 0.6086 - accuracy: 0.7046
Epoch 9/10
32/32 [==============================] - 0s 712u

## 드롭아웃 정규화를 사용하여 과접합 줄이기

In [18]:
# 최적 배치 크기와 에포크를 정하기 위한 그리드 탐색
# 필요한 패키지 임포트
from keras.layers import Dropout  # 임포트 라인에 추가 

# 랜덤 시드 정의
seed = 6
np.random.seed(seed)

# 모델 정의 
def create_model(learn_rate, dropout_rate): # 학습률과 드롭아웃 비율을 인자로
    # 카라스 모델 생성
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))        # 드롭아웃 레이어 추가 
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))        # 드롭아웃 레이어 추가   
    model.add(Dense(1, activation='sigmoid'))
    
    # 모델 컴파일 
    adam = Adam(lr = learn_rate)            # 학습률에 대한 변수 
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model
  
# 모델 생성 
model = KerasClassifier(build_fn = create_model, epochs = 50, 
                        batch_size = 10, verbose = 0)
                        

# 그리트 탐색 매개변수 정의
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# 그리드 탐색 매개변수를 딕셔너리로 변환 
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model, param_grid = param_grid, 
                    cv = KFold(random_state=seed, shuffle=True), 
                    verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................
[CV 1/5; 1/9] END dropout_rate=0.0, learn_rate=0.001;, score=0.772 total time=   1.2s
[CV 2/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................
[CV 2/5; 1/9] END dropout_rate=0.0, learn_rate=0.001;, score=0.709 total time=   1.3s
[CV 3/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................
[CV 3/5; 1/9] END dropout_rate=0.0, learn_rate=0.001;, score=0.769 total time=   1.2s
[CV 4/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................
[CV 4/5; 1/9] END dropout_rate=0.0, learn_rate=0.001;, score=0.821 total time=   1.2s
[CV 5/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................
[CV 5/5; 1/9] END dropout_rate=0.0, learn_rate=0.001;, score=0.769 total time=   1.1s
[CV 1/5; 2/9] START dropout_rate=0.0, learn_rate=0.01...........................
[CV 1/5; 2/9] END dropou

## 최적 초매개변수 찾기 

In [20]:
# activation, init 그리드 탐색 

# 랜덤 시드 지정
seed = 6
np.random.seed(seed)

# 모델 정의 
def create_model(activation, init):  # 인자 지정 
    # 모델 생성
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # 모델 컴파일 
    adam = Adam(lr = 0.001)   # 학습률을 하드코딩함.
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# 모델 생성
model = KerasClassifier(build_fn = create_model, epochs = 100, 
                        batch_size = 20, verbose = 0)

# 그리드 탐색 매개변수 정의 
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# 그리드 탐색 매개변수를 딕셔너리로 변환 
param_grid = dict(activation = activation, init = init)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model, param_grid = param_grid, 
                    cv = KFold(random_state=seed, shuffle=True), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고 
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation=softmax, init=uniform...........................
[CV 1/5; 1/12] END activation=softmax, init=uniform;, score=0.797 total time=   1.4s
[CV 2/5; 1/12] START activation=softmax, init=uniform...........................
[CV 2/5; 1/12] END activation=softmax, init=uniform;, score=0.696 total time=   1.3s
[CV 3/5; 1/12] START activation=softmax, init=uniform...........................
[CV 3/5; 1/12] END activation=softmax, init=uniform;, score=0.795 total time=   1.2s
[CV 4/5; 1/12] START activation=softmax, init=uniform...........................
[CV 4/5; 1/12] END activation=softmax, init=uniform;, score=0.795 total time=   1.2s
[CV 5/5; 1/12] START activation=softmax, init=uniform...........................
[CV 5/5; 1/12] END activation=softmax, init=uniform;, score=0.769 total time=   1.4s
[CV 1/5; 2/12] START activation=softmax, init=normal............................
[CV 1/5; 2/12] END activatio

## 뉴런의 개수 최적화하기

In [21]:
# 히든 레이어의 뉴런의 최적 개수를 찾기 위한 그리드 탐색 

# 랜덤 시드 설정 
seed = 6
np.random.seed(seed)

# 모델 정의 
def create_model(neuron1, neuron2):
    # 케라스 모델 생성
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, 
                    kernel_initializer= 'uniform', 
                    activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, 
                    kernel_initializer= 'uniform', 
                    activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 모델 컴파일
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = adam, 
                  metrics = ['accuracy'])
    return model

# 모델 생성
model = KerasClassifier(build_fn = create_model, 
                        epochs = 100, batch_size = 20, verbose = 0)

# 그리드 탐색 매개변수 설정 
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# 그리드 탐색 매개변수를 딕셔너리로 변화 
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model, 
                    param_grid = param_grid, 
                    cv = KFold(random_state=seed, shuffle=True), 
                    refit = True, 
                    verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.785 total time=   1.2s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.722 total time=   1.3s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.795 total time=   1.1s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.821 total time=   1.2s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.782 total time=   1.2s
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END .........neuron1=4, neuron2=4;,

## 최적의 초매개변수를 사용하여 예측하기

다음 코드와 같이 `predict()` 함수를 사용하여 데이터셋의 모든 사례에 대하여 당뇨병이 발생할지를 예측해 보자. 

In [22]:
# 최적 초매개변수를 가지고 예측값 생성
import numpy as np
y_pred = grid.predict(X_standardized)

/Users/brainiac/opt/anaconda3/envs/ml4hap/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


다음 코드와 같이 예측값의 형태(shape)을 알아보자. 

In [23]:
print(y_pred.shape)

(392, 1)


In [24]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [25]:
from sklearn.metrics import classification_report, accuracy_score
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7806122448979592
              precision    recall  f1-score   support

           0       0.81      0.89      0.84       262
           1       0.71      0.57      0.63       130

    accuracy                           0.78       392
   macro avg       0.76      0.73      0.74       392
weighted avg       0.77      0.78      0.77       392



## 보너스

In [26]:
example = df.iloc[1]
print(example)

n_pregnant                   0.000
glucose_concentration      137.000
blood_pressure (mm Hg)      40.000
skin_thickness (mm)         35.000
serum_insulin (mu U/ml)    168.000
BMI                         43.100
pedigree_function            2.288
age                         33.000
class                        1.000
Name: 4, dtype: float64


In [27]:
prediction = grid.predict(X_standardized[1].reshape(1, -1))
print(prediction)

[[1]]


/Users/brainiac/opt/anaconda3/envs/ml4hap/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## 요약

이 장에서 우리는 케라스를 사용하여 딥러닝 신경망을 만들고, 사이킷런 그리드 탐색법을 사용하여 최적의 초매개변수를 찾아보았다. 그리고 초매개변수를 조정하여 신경망을 최적화시키는 방법을 배웠다. 우리 모두 똑같은 결과를 얻지 못했을 수도 있지만, 대체로 비슷한 예측값을 얻을 수 있었다면 우리의 모델이 비교적 성공적이라고 생각할 수 있다. 새로운 데이터로 또는 새로운 데이터로 다른 문제를 접했을 때도 이와 비슷한 과정을 통해서 문제를 해결할 수 있다. 이 장에 우리는 딥러닝, 초매개변수 최적화 방법을 배웠고, 이것을 신경망에 적용하여 당뇨병 환자 데이터에서 당뇨병의 발명을 예측하는 데 응용할 수 있었다.

다음 장에서는 머신러닝을 사용하여 DNA를 분류하는 방법에 대해서 살펴볼 것이다.